In [4]:
# Add user specific python libraries to path
import sys
sys.path.insert(0, "/home/smehra/local-packages")

In [5]:
import numpy as np
import pandas as pd

import geopandas as gpd
from shapely.geometry import Point, Polygon

# enable automated generational garbage collection
import gc
gc.enable()

import matplotlib.pyplot as plt
%matplotlib inline

import time
from datetime import timedelta  
from datetime import date
from datetime import datetime

from os import listdir, environ, mkdir, utime


## Read Data

In [6]:
country_of_raw_data = 'ITA'

input_data_directories = ['/data/covid/FB_Data/Italy_mobility']

file_paths = []
for direc in input_data_directories:
    for file_name in listdir(direc):
        file_path = direc + '/' + file_name    
        file_paths.append(file_path)
    
# get data from files in csv format
data = []
dateparser = lambda x: datetime.strptime(x, '%Y-%m-%d %H%M')
for file_path in file_paths:
    file_data = pd.read_csv(file_path,
                            parse_dates = ['date_time'], 
                            date_parser = dateparser)
    file_data = file_data[['date_time', 'n_crisis', 'n_baseline', 'start_lat', 'start_lon', 'end_lat', 'end_lon']]
    data.append(file_data)

# concatenate all data
FB_data = pd.concat(data, ignore_index=True)
FB_data.head()


,date_time,n_crisis,n_baseline,start_lat,start_lon,end_lat,end_lon
0,2020-04-11 08:00:00,136,101.4,37.030706,9.972693,37.030706,9.972693
1,2020-04-11 08:00:00,122,71.4,43.420986,10.452991,43.420986,10.452991
2,2020-04-11 08:00:00,32,132.8,38.076287,13.497052,38.130259,13.330923
3,2020-04-11 08:00:00,10,60.2,38.076287,13.497052,38.130259,13.330923
4,2020-04-11 08:00:00,60,238.6,38.076287,13.497052,38.130259,13.330923


In [7]:
# get all unique lat - long pairs

start_points = FB_data[['start_lat', 'start_lon']].copy()
start_points.rename(columns = {'start_lat': 'latitude', 'start_lon': 'longitude'}, inplace = True)

end_points = FB_data[['end_lat', 'end_lon']].copy()
end_points.rename(columns = {'end_lat': 'latitude', 'end_lon': 'longitude'}, inplace = True)

all_points = pd.concat([start_points, end_points], ignore_index=True)
all_points.drop_duplicates(inplace = True)
all_points.reset_index(drop = True, inplace = True)

all_points = gpd.GeoDataFrame(all_points, geometry = gpd.points_from_xy(all_points.longitude, all_points.latitude))
all_points.crs = {'init' :'epsg:4326'}
all_points.head()


,latitude,longitude,geometry
0,37.030706,9.972693,POINT (9.972693237979801 37.030705925116)
1,43.420986,10.452991,POINT (10.452990611038 43.420986266513)
2,38.076287,13.497052,POINT (13.497051536203 38.07628723147)
3,41.469484,13.488840,POINT (13.488840351018 41.469483822716)
4,41.955105,13.476910,POINT (13.476910244718 41.955104787927)


## Get states for each point

In [8]:
# read adm1 shapefile
states = gpd.read_file('/data/tmp/smehra/tmp/gpl-covid/data/interim/adm/adm1/adm1.shp')
# keep only country of interest
states = states[states.adm0_name == country_of_raw_data]
# drop administrative units with no geometry
states = states[states.geometry.notna()]
# sort by state
states.sort_values(['adm1_name'], inplace = True)
states.reset_index(drop=True, inplace = True)
states.head()


,adm0_name,adm1_name,latitude,longitude,name_alt,population,area_km2,pop_densit,geometry
0,ITA,Abruzzo,42.228034,13.855137,None,1311580.0,19608.424998,66.888595,"POLYGON ((14.25307083129883 41.88950347900391,..."
1,ITA,Basilicata,40.500675,16.082277,None,562869.0,17227.281961,32.673117,(POLYGON ((16.72180366516125 40.21236038208013...
2,ITA,Calabria,39.068132,16.347359,None,1947131.0,24947.307609,78.049745,(POLYGON ((15.80208206176763 39.70013809204102...
3,ITA,Campania,40.859986,14.840554,None,5801692.0,23769.153908,244.084919,(POLYGON ((14.25263881683355 40.54152679443371...
4,ITA,Emilia Romagna,44.536529,11.019991,None,4459477.0,43354.056242,102.861817,(POLYGON ((12.41045093536383 43.89839553833008...


In [9]:
# create a "points with states" results dataset
points_with_states = all_points.copy()

# iterate through all poi's within that state
for point in all_points.itertuples():

    # iterate through all counties within that state
    for state in states.itertuples():

        # check if county countains the poi
        if state.geometry.contains(point.geometry):

            # update results database
            points_with_states.at[point.Index, 'state'] = state.adm1_name
            break

points_with_states.head()

,latitude,longitude,geometry,state
0,37.030706,9.972693,POINT (9.972693237979801 37.030705925116),NaN
1,43.420986,10.452991,POINT (10.452990611038 43.420986266513),Toscana
2,38.076287,13.497052,POINT (13.497051536203 38.07628723147),Sicilia
3,41.469484,13.488840,POINT (13.488840351018 41.469483822716),Lazio
4,41.955105,13.476910,POINT (13.476910244718 41.955104787927),Abruzzo


## Get counties for each point

In [10]:
# read adm2 shapefile
counties = gpd.read_file('/data/tmp/smehra/tmp/gpl-covid/data/interim/adm/adm2/adm2.shp')
# keep only country of interest
counties = counties[counties.adm0_name == country_of_raw_data]
# drop administrative units with no geometry
counties = counties[counties.geometry.notna()]
# sort by state
counties.sort_values(['adm1_name', 'adm2_name'], inplace = True)
counties.reset_index(drop=True, inplace = True)
counties.head()


,adm0_name,adm1_name,adm2_name,latitude,longitude,name_alt,population,area_km2,pop_densit,geometry
0,ITA,Abruzzo,Chieti,42.101086,14.382700,Abruzos|Abruzzen|Abruzzes|Abruzzi,385588.0,4676.632198,82.449931,"POLYGON ((14.25307083129883 41.88950347900391,..."
1,ITA,Abruzzo,L'Aquila,42.104988,13.603098,Abruzos|Abruzzen|Abruzzes|Abruzzi,299031.0,9118.912823,32.792396,"POLYGON ((13.32633399963385 41.94295120239269,..."
2,ITA,Abruzzo,Pescara,42.332625,13.984460,Abruzos|Abruzzen|Abruzzes|Abruzzi,318909.0,2232.424943,142.853179,"POLYGON ((13.80143928527838 42.19025421142572,..."
3,ITA,Abruzzo,Teramo,42.645867,13.726467,Abruzos|Abruzzen|Abruzzes|Abruzzi,308052.0,3580.455034,86.037109,"POLYGON ((14.09513854980474 42.57933044433605,..."
4,ITA,Basilicata,Matera,40.470248,16.446244,Basilicate|Lucania,197909.0,5934.840025,33.346981,(POLYGON ((16.72180366516125 40.21236038208013...


In [11]:
# create a results dataset
points_with_adm_info = points_with_states.copy()

# get a sorted list of unique states
states = sorted(points_with_states[points_with_states.state.notna()].state.unique())

for state in states:
    
    # get all relevant counties within the state
    relevant_counties = counties[counties.adm1_name == state]
    relevant_points = points_with_states[(points_with_states.state == state)]
    
    print(datetime.now().strftime("%d/%m/%Y %H:%M:%S") + ' Current State: ' + state)
    print(datetime.now().strftime("%d/%m/%Y %H:%M:%S") + ' Number of points: ' + str(len(relevant_points)))

    # iterate through all poi's within that state
    for point in relevant_points.itertuples():
        
        # iterate through all counties within that state
        for county in relevant_counties.itertuples():
            
            # check if county countains the poi
            if county.geometry.contains(point.geometry):
                
                # update results database
                points_with_adm_info.at[point.Index, 'county'] = county.adm2_name
                break
                
points_with_adm_info.head()

21/05/2020 19:45:19 Current State: Abruzzo
21/05/2020 19:45:19 Number of points: 157
21/05/2020 19:45:19 Current State: Basilicata
21/05/2020 19:45:19 Number of points: 98
21/05/2020 19:45:19 Current State: Calabria
21/05/2020 19:45:19 Number of points: 191
21/05/2020 19:45:20 Current State: Campania
21/05/2020 19:45:20 Number of points: 217
21/05/2020 19:45:21 Current State: Emilia Romagna
21/05/2020 19:45:21 Number of points: 259
21/05/2020 19:45:21 Current State: Friuli Venezia Giulia
21/05/2020 19:45:21 Number of points: 126
21/05/2020 19:45:22 Current State: Lazio
21/05/2020 19:45:22 Number of points: 208
21/05/2020 19:45:22 Current State: Liguria
21/05/2020 19:45:22 Number of points: 92
21/05/2020 19:45:23 Current State: Lombardia
21/05/2020 19:45:23 Number of points: 445
21/05/2020 19:45:23 Current State: Marche
21/05/2020 19:45:23 Number of points: 131
21/05/2020 19:45:24 Current State: Molise
21/05/2020 19:45:24 Number of points: 68
21/05/2020 19:45:24 Current State: Piemonte


,latitude,longitude,geometry,state,county
0,37.030706,9.972693,POINT (9.972693237979801 37.030705925116),NaN,NaN
1,43.420986,10.452991,POINT (10.452990611038 43.420986266513),Toscana,Livorno
2,38.076287,13.497052,POINT (13.497051536203 38.07628723147),Sicilia,Palermo
3,41.469484,13.488840,POINT (13.488840351018 41.469483822716),Lazio,Frosinone
4,41.955105,13.476910,POINT (13.476910244718 41.955104787927),Abruzzo,L'Aquila


## Compute source destination metrics aggregated at county level

In [12]:
# merge with points_with_adm_info to get start counties
start_data = points_with_adm_info[['latitude','longitude','state','county']].copy()
start_data.columns = ['start_lat','start_lon', 'start_adm1_name', 'start_adm2_name']
FB_data_with_adm = FB_data.merge(start_data, on = ['start_lat','start_lon'], how = 'left')

# merge with points_with_adm_info to get end counties
end_data = points_with_adm_info[['latitude','longitude','state','county']].copy()
end_data.columns = ['end_lat','end_lon', 'end_adm1_name', 'end_adm2_name']
FB_data_with_adm = FB_data_with_adm.merge(end_data, on = ['end_lat','end_lon'], how = 'left')

FB_data_with_adm.head()

,date_time,n_crisis,n_baseline,start_lat,start_lon,end_lat,end_lon,start_adm1_name,start_adm2_name,end_adm1_name,end_adm2_name
0,2020-04-11 08:00:00,136,101.4,37.030706,9.972693,37.030706,9.972693,NaN,NaN,NaN,NaN
1,2020-04-11 08:00:00,122,71.4,43.420986,10.452991,43.420986,10.452991,Toscana,Livorno,Toscana,Livorno
2,2020-04-11 08:00:00,32,132.8,38.076287,13.497052,38.130259,13.330923,Sicilia,Palermo,Sicilia,Palermo
3,2020-04-11 08:00:00,10,60.2,38.076287,13.497052,38.130259,13.330923,Sicilia,Palermo,Sicilia,Palermo
4,2020-04-11 08:00:00,60,238.6,38.076287,13.497052,38.130259,13.330923,Sicilia,Palermo,Sicilia,Palermo


In [13]:
source_destination_metrics = FB_data_with_adm.copy()

source_destination_metrics['date'] = source_destination_metrics.date_time.dt.date
source_destination_metrics = source_destination_metrics.groupby(['date', 
                                                'start_adm1_name', 
                                                'start_adm2_name', 
                                                'end_adm1_name', 
                                                'end_adm2_name']).agg({'n_crisis': 'sum', 'n_baseline': 'sum'})

source_destination_metrics.reset_index(inplace = True)
source_destination_metrics.head(20)

,date,start_adm1_name,start_adm2_name,end_adm1_name,end_adm2_name,n_crisis,n_baseline
0,2020-02-23,Abruzzo,Chieti,Abruzzo,Chieti,44358,44391.8
1,2020-02-23,Abruzzo,Chieti,Abruzzo,Pescara,1197,1189.4
2,2020-02-23,Abruzzo,L'Aquila,Abruzzo,L'Aquila,42602,41545.0
3,2020-02-23,Abruzzo,L'Aquila,Abruzzo,Pescara,30,24.6
4,2020-02-23,Abruzzo,L'Aquila,Campania,Napoli,311,493.8
5,2020-02-23,Abruzzo,L'Aquila,Lazio,Rieti,45,34.0
6,2020-02-23,Abruzzo,L'Aquila,Lazio,Roma,66,70.0
7,2020-02-23,Abruzzo,Pescara,Abruzzo,Chieti,1308,1277.6
8,2020-02-23,Abruzzo,Pescara,Abruzzo,L'Aquila,13,11.0
9,2020-02-23,Abruzzo,Pescara,Abruzzo,Pescara,41869,41907.2


In [14]:
source_destination_metrics.to_csv('/data/tmp/smehra/aggregated_data/gpl-covid-mobility/FB_' + country_of_raw_data + '_source_destination_metrics.csv', index = False)


## Compute county relative metrics

In [15]:
# lean up source distance metrics dataset with county name columns

source_dest = source_destination_metrics.copy()
source_dest['start_county'] = source_destination_metrics.start_adm1_name + ' - ' + source_destination_metrics.start_adm2_name
source_dest['end_county'] = source_destination_metrics.end_adm1_name + ' - ' + source_destination_metrics.end_adm2_name
source_dest = source_dest[['date', 'start_county', 'end_county', 'n_crisis', 'n_baseline']]
source_dest.head()


,date,start_county,end_county,n_crisis,n_baseline
0,2020-02-23,Abruzzo - Chieti,Abruzzo - Chieti,44358,44391.8
1,2020-02-23,Abruzzo - Chieti,Abruzzo - Pescara,1197,1189.4
2,2020-02-23,Abruzzo - L'Aquila,Abruzzo - L'Aquila,42602,41545.0
3,2020-02-23,Abruzzo - L'Aquila,Abruzzo - Pescara,30,24.6
4,2020-02-23,Abruzzo - L'Aquila,Campania - Napoli,311,493.8


In [16]:
# get list of all unique counties

start_counties = source_dest.start_county
end_counties = source_dest.end_county

all_counties = pd.concat([start_counties, end_counties], ignore_index=True)
all_counties.drop_duplicates(inplace = True)
all_counties.reset_index(drop = True, inplace = True)
all_counties.head()


0       Abruzzo - Chieti
1     Abruzzo - L'Aquila
2      Abruzzo - Pescara
3       Abruzzo - Teramo
4    Basilicata - Matera
dtype: object

In [17]:
# create a results dataset
county_relative_movement = pd.DataFrame(columns=['county', 'date', 'baseline_within_movement',
                                                 'baseline_enter_movement', 'baseline_exit_movement',
                                                 'crisis_within_movement', 'crisis_enter_movement',
                                                 'crisis_exit_movement'])

# compute metrics for each date
for current_date in source_dest.date.unique().tolist():
        
    # compute metrics for each county
    for county in all_counties:
        
        baseline_within = source_dest[(source_dest.date == current_date) & 
                                      (source_dest.start_county == county) & 
                                      (source_dest.end_county == county)].n_baseline.sum()
                
        baseline_enter = source_dest[(source_dest.date == current_date) & 
                                     (source_dest.start_county != county) & 
                                     (source_dest.end_county == county)].n_baseline.sum()
    
        baseline_exit = source_dest[(source_dest.date == current_date) & 
                                    (source_dest.start_county == county) & 
                                    (source_dest.end_county != county)].n_baseline.sum()

        crisis_within = source_dest[(source_dest.date == current_date) & 
                                    (source_dest.start_county == county) & 
                                    (source_dest.end_county == county)].n_crisis.sum()
        
        crisis_enter = source_dest[(source_dest.date == current_date) & 
                                   (source_dest.start_county != county) & 
                                   (source_dest.end_county == county)].n_crisis.sum()
        
        crisis_exit = source_dest[(source_dest.date == current_date) & 
                                  (source_dest.start_county == county) & 
                                  (source_dest.end_county != county)].n_crisis.sum()
        
        county_relative_movement = county_relative_movement.append({'county': county, 
                                                                    'date': current_date, 
                                                                    'baseline_within_movement': baseline_within,
                                                                    'baseline_enter_movement': baseline_enter,
                                                                    'baseline_exit_movement': baseline_exit,
                                                                    'crisis_within_movement': crisis_within,
                                                                    'crisis_enter_movement': crisis_enter,
                                                                    'crisis_exit_movement': crisis_exit
                                                                   }, ignore_index=True)

county_relative_movement.head()


,county,date,baseline_within_movement,baseline_enter_movement,baseline_exit_movement,crisis_within_movement,crisis_enter_movement,crisis_exit_movement
0,Abruzzo - Chieti,2020-02-23,44391.8,1277.6,1189.4,44358,1308,1197
1,Abruzzo - L'Aquila,2020-02-23,41545.0,115.0,622.4,42602,89,452
2,Abruzzo - Pescara,2020-02-23,41907.2,1284.0,1352.8,41869,1280,1384
3,Abruzzo - Teramo,2020-02-23,38704.4,578.4,604.2,39041,650,744
4,Basilicata - Matera,2020-02-23,25152.8,72.6,97.2,25539,67,100


In [18]:
county_relative_movement['adm1_name'] = county_relative_movement.county.apply(lambda x: x.split('-')[0][:-1])
county_relative_movement['adm2_name'] = county_relative_movement.county.apply(lambda x: x.split('-')[1][1:])

county_relative_movement = county_relative_movement[['adm1_name', 'adm2_name', 'date', 
                                                     'baseline_within_movement', 'baseline_enter_movement', 
                                                     'baseline_exit_movement', 'crisis_within_movement',
                                                     'crisis_enter_movement', 'crisis_exit_movement']]

county_relative_movement.sort_values(['adm1_name', 'adm2_name', 'date'], inplace = True)   
county_relative_movement.reset_index(inplace = True, drop = True)
county_relative_movement.head()


,adm1_name,adm2_name,date,baseline_within_movement,baseline_enter_movement,baseline_exit_movement,crisis_within_movement,crisis_enter_movement,crisis_exit_movement
0,Abruzzo,Chieti,2020-02-23,44391.80,1277.60,1189.40,44358,1308,1197
1,Abruzzo,Chieti,2020-02-24,44840.40,1909.60,1985.40,45339,1822,1928
2,Abruzzo,Chieti,2020-02-25,45385.25,1977.75,1946.25,45808,1806,1785
3,Abruzzo,Chieti,2020-02-26,45452.20,1954.20,1941.80,46121,1849,1841
4,Abruzzo,Chieti,2020-02-27,45256.00,1955.60,2013.80,46187,1912,1968


In [19]:
county_relative_movement.to_csv('/data/tmp/smehra/aggregated_data/gpl-covid-mobility/FB_' + country_of_raw_data + '_county_relative_movement.csv', index = False)        
